In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pandas_ods_reader -q
from pandas_ods_reader import read_ods

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
!pip3 install pandas_ods_reader -q
from pandas_ods_reader import read_ods

In [ ]:
# data_sentiment = read_ods('/content/hi_3500.ods',1,headers = False)
# data_sentiment.columns = ['text','sentiment']
# data_sentiment.head()

In [ ]:
data_sentiment = read_ods('/content/drive/MyDrive/NLP Project/Sentiment data/hi_3500.ods',1,headers = False)
data_sentiment.columns = ['text','sentiment']

data_sentiment.head()

,text,sentiment
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [ ]:
data_sentiment.dropna()
# data_sentiment_test.dropna()

,text,sentiment
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
...,...,...
9072,RT @aajtak: #Mojo बजरंग बली का सबसे विशालकाय र...,neutral
9073,RT @aajtak: कहां अचानक स्कूल में पड़ा मंत्री क...,neutral
9074,RT @aajtak: #MoJo सुनिए कुलभूषण की दर्दभरी आपब...,neutral
9075,RT @aajtak: लव जेहाद के नाम पर क्या उत्तर प्रद...,neutral


In [ ]:
# Define a function to remove English words, numeric characters, and specific special characters
def clean_hindi_text(text):
    words = nltk.word_tokenize(text)
    cleaned_words = []
    for word in words:
        if not re.match("^[A-Za-z]*$", word) and not bool(re.search(r'[0-9०-९]', word)):
            # Specify special characters to remove (e.g., @, ., ,, #, &)
            word = re.sub(r'[.,@#&:)(]', '', word)
            cleaned_words.append(word)
    return ' '.join(cleaned_words)

In [ ]:
data_sentiment['text'] = data_sentiment['text'].apply(clean_hindi_text)
# data_sentiment_test['text'] = data_sentiment_test['text'].apply(clean_hindi_text)
data_sentiment

,text,sentiment
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
...,...,...
9072,बजरंग बली का सबसे विशालकाय रूप कीजिए ' साक...,neutral
9073,कहां अचानक स्कूल में पड़ा मंत्री का छापा ? फ...,neutral
9074,सुनिए कुलभूषण की दर्दभरी आपबीती अन्य वीडिय...,neutral
9075,लव जेहाद के नाम पर क्या उत्तर प्रदेश में हो ...,neutral


In [ ]:
X = data_sentiment['text']
y = data_sentiment['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [ ]:
# # Split the dataset into training and testing sets
# X_train = data_sentiment_train['text']  # Your Hindi text data
# X_test = data_sentiment_test['text']
# y_train = data_sentiment_train['sentiment']  # Sentiment labels (e.g., positive, negative)
# y_test = data_sentiment_test['sentiment']

In [ ]:
# Create TF-IDF vectors from the text data
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
from sklearn.svm import SVC

# Initialize and train a Support Vector Machine classifier for multiclass classification
svm_classifier = SVC(kernel='linear', decision_function_shape='ovr', random_state=42)
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment on the test data
y_pred_svm = svm_classifier.predict(X_test_tfidf)

# Evaluate the SVM model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

# Print evaluation metrics for SVM
print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {accuracy_svm:.2f}")
print(classification_rep_svm)


Support Vector Machine (SVM) Classifier:
Accuracy: 0.71
              precision    recall  f1-score   support

    negative       0.68      0.74      0.71      1065
     neutral       0.75      0.69      0.72       886
    positive       0.70      0.69      0.70      1045

    accuracy                           0.71      2996
   macro avg       0.71      0.71      0.71      2996
weighted avg       0.71      0.71      0.71      2996



In [ ]:
# # Initialize and train a Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train_tfidf, y_train)
# # Predict sentiment on the test data
# y_pred = rf_classifier.predict(X_test_tfidf)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# classification_rep = classification_report(y_test, y_pred)

# # Print evaluation metrics
# print(f"Accuracy: {accuracy:.2f}")
# print(classification_rep)

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier

# # Initialize and train a Gradient Boosting classifier
# gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
# gb_classifier.fit(X_train_tfidf, y_train)

# # Predict sentiment on the test data
# y_pred_gb = gb_classifier.predict(X_test_tfidf)

# # Evaluate the Gradient Boosting model
# accuracy_gb = accuracy_score(y_test, y_pred_gb)
# classification_rep_gb = classification_report(y_test, y_pred_gb)

# # Print evaluation metrics for Gradient Boosting
# print("Gradient Boosting Classifier:")
# print(f"Accuracy: {accuracy_gb:.2f}")
# print(classification_rep_gb)


In [ ]:
# from sklearn.naive_bayes import MultinomialNB

# # Initialize and train a Multinomial Naive Bayes classifier
# nb_classifier = MultinomialNB()
# nb_classifier.fit(X_train_tfidf, y_train)

# # Predict sentiment on the test data
# y_pred_nb = nb_classifier.predict(X_test_tfidf)

# # Evaluate the Naive Bayes model
# accuracy_nb = accuracy_score(y_test, y_pred_nb)
# classification_rep_nb = classification_report(y_test, y_pred_nb)

# # Print evaluation metrics for Naive Bayes
# print("Multinomial Naive Bayes Classifier:")
# print(f"Accuracy: {accuracy_nb:.2f}")
# print(classification_rep_nb)


*italicized text*# SVC on Fake news data for sentiment

In [ ]:
# train = pd.read_csv('/content/train (5).csv',encoding='utf-8-sig')
# test = pd.read_csv('/content/test (2).csv',encoding='utf-8-sig')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP Project/Data/train.csv',encoding='utf-8-sig')
test = pd.read_csv('/content/drive/MyDrive/NLP Project/Data/test.csv',encoding='utf-8-sig')

In [ ]:
train.drop(['Unnamed: 0','author','id'], inplace = True, axis = 1)
test.drop(['Unnamed: 0','author','id'], inplace = True, axis = 1)

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
hindi_stopwords = [
    'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'इस', 'इसके', 'इसको', 'इसमें', 'इससे',
    'उनका', 'उनकी', 'उनके', 'उनको', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हें', 'उसका',
    'उसकी', 'उसके', 'उसको', 'उसमें', 'उसने', 'उसी', 'उसे', 'उसका', 'करता', 'करते',
    'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'कारण', 'किसी', 'की', 'कुछ', 'के',
    'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन',
    'जिन्होंने', 'जिन्हों', 'जिस', 'जिसे', 'जी', 'जिसके', 'जिसको', 'जिसमें', 'जिसे', 'तक',
    'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'था', 'थे', 'दिया', 'दूसरे',
    'दो', 'द्वारा', 'ने', 'पर', 'पहले', 'पूरा', 'पूरी', 'फिर', 'बनी', 'बहुत', 'बाद',
    'बाला', 'भी', 'मगर', 'मानो', 'मैं', 'मैंने', 'में', 'यदि', 'यह', 'यहाँ', 'यही',
    'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'लिए', 'लिया', 'लेकिन', 'व', 'वगेरह', 'वर्ग',
    'वह', 'वहाँ', 'वही', 'वाले', 'वाली', 'वाला', 'वाले', 'वाली', 'वाला', 'वाले', 'वाली',
    'वाला', 'सबसे', 'सबसे', 'सकता', 'सकते', 'सकती', 'सकती', 'सब', 'सबसे', 'से', 'ही',
    'है', 'हैं', 'हुआ', 'हुई', 'हुए', 'हो', 'होता', 'होते', 'होती', 'होती', 'होते', 'होते',
    'होती', 'होती', 'होती', 'होना', 'होने', 'अंदर', 'अदि', 'अन्य', 'अप', 'अफ', 'अल', 'अलब',
    'अव', 'अवश्य', 'अस', 'आदि', 'आप', 'आपका', 'आपकी', 'आपके', 'आपने', 'आपने', 'आदि', 'आना',
    'इंहोंने', 'इंहें', 'इंहों', 'इतना', 'इतनी', 'इतने', 'इन', 'इनका', 'इनकी', 'इनके', 'इन्ही',
    'इन्होंने', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसी', 'उंहोंने', 'उंहें',
    'उंहों', 'उपर', 'उसके', 'उससे', 'उसी', 'उसे', 'उसी', 'उसे', 'कर', 'करत', 'करता', 'करती',
    'करते', 'करन', 'करना', 'करने', 'किया', 'किये', 'किये', 'किया', 'कुछ', 'कुल', 'के', 'को', 'को'
]


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download("indian")

nltk.download("stopwords")
def remove_hindi_stopwords(text):
    words = nltk.word_tokenize(text)
    cleaned_words = []
    for word in words:
        # Remove specific words
        if word.lower() not in hindi_stopwords:
            # Remove English words and special characters
            if not re.match("^[A-Za-z]*$", word) and not bool(re.search(r'[0-9०-९]', word)):
                # Specify special characters to remove (e.g., @, ., ,, #, &)
                word = re.sub(r'[.,@#&]', '', word)
                cleaned_words.append(word)
    return ' '.join(cleaned_words)



[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train['text'] = train['text'].apply(remove_hindi_stopwords)
test['text'] = test['text'].apply(remove_hindi_stopwords)

In [ ]:
# Create TF-IDF vectors from the text data
#tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
train_tfidf = tfidf_vectorizer.transform(train['text'])
test_tfidf = tfidf_vectorizer.transform(test['text'])

In [ ]:
sentim_train = svm_classifier.predict(train_tfidf)
sentim_test = svm_classifier.predict(test_tfidf)

In [ ]:
train['sentiment'] = sentim_train
test['sentiment'] = sentim_test

In [ ]:
train['sentiment'].value_counts()

-1    1491
 1     318
 0     246
Name: sentiment, dtype: int64

In [ ]:
sentiment_mapping = {"negative": -1, "neutral": 0, "positive": 1}
train['sentiment'] = train['sentiment'].map(sentiment_mapping)
test['sentiment'] = test['sentiment'].map(sentiment_mapping)

In [ ]:
row_with_sentiment_zero = train.loc[train['sentiment'] == 0].iloc[2]
row_with_sentiment_zero.text

"मैसाचुसेट्स कॉप पत्नी ब्लैक लिव्स मैटर स्रोत पिनक फेक होम-इन्वेस्टमेंट रॉबरी बस्टेड : पिनैक एक मैसाचुसेट्स पुलिस पत्नी झूठे पुलिस रिपोर्ट दर्ज आरोप लग  क्योंकि एक चोर घुसकर तोड़फोड़  गहने डॉलर चोरी और ब्लैक लाइव्स मैटर साथ ब्लैक लोगों पिन कहानी सुनाई। मिलबरी सिपाही डैनियल डेली पत्नी मारिया डैली अक्टूबर पुलिस रिपोर्ट दर्ज सपने शिकार बारे सोशल मीडिया लिया। बोस्टन हेराल्ड अनुसार  `` हम सो  न केवल हमारे लूट  बल्कि नफरत बिना देखने जाग था। '' अगर निजी फेसबुक अकाउंट आरोप लगाते  `` अगर [ ] मुझसे कल ब्लू लाइफ और ब्लैक लाइफ बारे पूछते  तो मेरी प्रतिक्रिया सकारात्मक [ ] होगी। '' “ आज दूसरी तरफ मुझे गुस्सा और नफरत कि जैसा नहीं हूँ। जो हमें दिनों निपटना और मुझे बीमार कि जो मेरे तरफ था। ” मारिया डैली पुलिस बताने बुलाया कि लूट सो रही थी और ब्लैक लिव्स मैटर बीएलएम साथ स्प्रे-पेंट और सोशल मीडिया बारे पोस्ट किया। मिलबरी पुलिस प्रमुख डोनाल्ड डेसोरसी हेराल्ड बताया कि डेली अक्टूबर सुबह-सुबह पुलिस रिपोर्ट बुलाया  दावा कि बाहर हजारों डॉलर मूल्य कीमती सामान और स्प्रे-पेंटिंग `` बीएलएम '' साथ बंद था। चीफ डेसो

Fake news detection

In [ ]:
#train.head()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler


In [ ]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
pipeline_rfc = Pipeline([
    # ('selector', ItemSelector(key='text')),
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', RandomForestClassifier())
])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC

pipeline_svc = Pipeline([
    # ('selector', ItemSelector(key='text')),
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC())
])

In [ ]:
# Split the dataset into training and testing sets
X = train['text']
y_sentiment = train['sentiment']
y_authenticity = train['label']

X_train, X_test, y_sentiment_train, y_sentiment_test, y_authenticity_train, y_authenticity_test = train_test_split(
    X, y_sentiment, y_authenticity, test_size=0.2, random_state=42
)

In [ ]:
y_sentiment_test

1540   -1
554     0
1451   -1
887    -1
1167    0
       ..
1907    0
1520    1
1440   -1
68     -1
1032   -1
Name: sentiment, Length: 411, dtype: int64

In [ ]:
svc_sentiment_test

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  0, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  0,  1,
       -1, -1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline_rfc_sent.fit(X_train, y_sentiment_train)
rfc_sentiment_train = pipeline_rfc_sent.predict(X_train)

rfc_sentiment_test = pipeline_rfc_sent.predict(X_test)

print(f"Accuracy for sentiments with RFC training:{accuracy_score(y_sentiment_train, rfc_sentiment_train)}")
print(f"Accuracy for sentiments with RFC test:{accuracy_score(y_sentiment_test, rfc_sentiment_test)}\n")

print(f"Classification rep for sentiments with RFC training\n:{classification_report(y_sentiment_train, rfc_sentiment_train)}")
print(f"Classification rep for sentiments with RFC test\n:{classification_report(y_sentiment_test, rfc_sentiment_test)}")

# print()

# pipeline_rfc_sent.fit(X_train,y_authenticity_train)
# rfc_authenticity_train = pipeline_rfc_sent.predict(X_train)

# rfc_authenticity_test = pipeline_rfc_sent.predict(X_test)

# print(f"Accuracy for news with RFC training:{accuracy_score(y_authenticity_train, rfc_authenticity_train)}")
# print(f"Accuracy for news with RFC test:{accuracy_score(y_authenticity_test, rfc_authenticity_test)}\n")

# print(f"Classification rep for news with RFC training\n:{classification_report(y_authenticity_train, rfc_authenticity_train)}")
# print(f"Classification rep for news with RFC test\n:{classification_report(y_authenticity_test, rfc_authenticity_test)}")



Accuracy for sentiments with RFC training:1.0
Accuracy for sentiments with RFC test:0.8491484184914841

Classification rep for sentiments with RFC training
:              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      1184
           0       1.00      1.00      1.00       194
           1       1.00      1.00      1.00       266

    accuracy                           1.00      1644
   macro avg       1.00      1.00      1.00      1644
weighted avg       1.00      1.00      1.00      1644

Classification rep for sentiments with RFC test
:              precision    recall  f1-score   support

          -1       0.83      1.00      0.91       307
           0       1.00      0.02      0.04        52
           1       0.98      0.81      0.88        52

    accuracy                           0.85       411
   macro avg       0.94      0.61      0.61       411
weighted avg       0.87      0.85      0.79       411



In [ ]:
pipeline_svc.fit(X_train, y_sentiment_train)
svc_sentiment_train = pipeline_svc.predict(X_train)

svc_sentiment_test = pipeline_svc.predict(X_test)

print(f"Accuracy for sentiments with RFC training:{accuracy_score(y_sentiment_train, svc_sentiment_train)}")
print(f"Accuracy for sentiments with RFC test:{accuracy_score(y_sentiment_test, svc_sentiment_test)}\n")

print(f"Classification rep for sentiments with RFC training\n:{classification_report(y_sentiment_train, svc_sentiment_train)}")
print(f"Classification rep for sentiments with RFC test\n:{classification_report(y_sentiment_test, svc_sentiment_test)}")
pipeline_svc.fit(X_train,y_authenticity_train)
svc_authenticity_train = pipeline_svc.predict(X_train)

svc_authenticity_test = pipeline_svc.predict(X_test)

print(f"Accuracy for news with SVC training:{accuracy_score(y_authenticity_train, svc_authenticity_train)}")
print(f"Accuracy for news with SVC test:{accuracy_score(y_authenticity_test, svc_authenticity_test)}\n")

print(f"Classification rep for news with SVC training\n:{classification_report(y_authenticity_train, svc_authenticity_train)}")
print(f"Classification rep for news with SVC test\n:{classification_report(y_authenticity_test, svc_authenticity_test)}")



Accuracy for sentiments with RFC training:0.9586374695863747
Accuracy for sentiments with RFC test:0.8491484184914841

Classification rep for sentiments with RFC training
:              precision    recall  f1-score   support

          -1       0.95      1.00      0.97      1184
           0       1.00      0.72      0.84       194
           1       0.99      0.95      0.97       266

    accuracy                           0.96      1644
   macro avg       0.98      0.89      0.93      1644
weighted avg       0.96      0.96      0.96      1644

Classification rep for sentiments with RFC test
:              precision    recall  f1-score   support

          -1       0.84      0.99      0.91       307
           0       1.00      0.04      0.07        52
           1       0.93      0.83      0.88        52

    accuracy                           0.85       411
   macro avg       0.92      0.62      0.62       411
weighted avg       0.87      0.85      0.80       411

Accuracy for news

In [ ]:
pipeline_svc.predict(["हाउस डिम एड: हमने जेसन चैफेट के ट्वीट करने तक कॉमी का पत्र भी नहीं देखा,हाउस डिम एड : हमने अक्टूबर  डारेल ल्यूकस इसे कॉमी पत्र नहीं देखा  अमेरिकन फोर्क  यूटा स्टैम्प जेसन शैफेट्ज सदस्यता लें ( छवि सौजन्य माइकल लॉली  एक क्रिएटिव कॉमन्स-बाय लाइसेंस तहत उपलब्ध ) कीथ ओल्बरमैन माफी साथ  संदेह नहीं कि विश्व खराब व्यक्ति सप्ताह है-एफबीआई निदेशक जेम्स कॉमी। एक हाउस डेमोक्रेटिक सहयोगी अनुसार  ऐसा लगता कि हम जानते कि दूसरा खराब व्यक्ति है। पता चला कि कॉमी अब-कुख्यात पत्र घोषणा कि एफबीआई उन ईमेलों देख जो हिलेरी क्लिंटन ईमेल सर्वर संबंधित  तो संबंधित समितियों रैंकिंग डेमोक्रेट कॉमी बारे सुना नहीं है। रिपब्लिकन कमेटी एक अध्यक्ष एक ट्वीट माध्यम पता चला। जैसा कि अब हम जानते  कॉमी रिपब्लिकन अध्यक्षों और हाउस इंटेलिजेंस  ज्यूडिशियरी  और ओवरसाइट समितियों डेमोक्रेटिक रैंकिंग सदस्यों सूचित कि एजेंसी उन ईमेलों समीक्षा रही थी जिन्हें हाल पता चला कि क्या वे वर्गीकृत जानकारी रखते हैं। पत्र बाहर जाने लंबे समय  ओवरसाइट कमेटी अध्यक्ष जेसन चैफेट्ज़ ट्वीट साथ राजनीतिक दुनिया आग लगा दी। एफबीआई डर मुझे सूचित  `` एफबीआई उन ईमेलों अस्तित्व बारे सीखा जो जांच अनुसार प्रतीत हैं। '' मामला खुल - जेसन शैफेट्ज़ (  ) अक्टूबर  बेशक  अब हम जानते कि मामला नहीं था। कॉमी वास्तव कह कि `` एक असंबंधित मामला '' प्रकाश ईमेल समीक्षा  जो अब हम एक किशोर साथ एंथोनी वेनर सेक्सटिंग जानते हैं। स्पष्ट रूप छोटी चीजें तथ्य रूप महत्वपूर्ण नहीं थीं। यूटा रिपब्लिकन जांच शुरू कसम खाई थी कि अगर हिलेरी कम कम साल जीतती  और संभवत : पूरे कार्यकाल लायक हैं। जाहिर तौर शैफेट्ज लगा कि एफबीआई काम रही - परिणामस्वरूप एक ट्वीट देश हल्का  कि कूलर प्रमुखों एहसास कि एक कठिन घटना है। एक वरिष्ठ हाउस डेमोक्रेटिक सहयोगी अनुसार  उस पत्र गलत तरीके फैलाना शायद चैफेट पापों कम कम हो। उस सहयोगी शेयरब्लू बताया कि बॉस और डेमोक्रेट उस समय कॉमी पत्र बारे नहीं पता - और केवल तभी पता चला उन्होंने ट्विटर जाँच की। `` संबंधित समितियों डेमोक्रेटिक रैंकिंग सदस्यों रिपब्लिकन अध्यक्षों कॉमी पत्र नहीं मिला। वास्तव डेमोक्रेटिक रैंकिंग सदस्यों इसे प्राप्त नहीं कि ओवरसाइट और सरकारी सुधार समिति अध्यक्ष  जेसन चैफेट्ज़ इसे ट्वीट नहीं और इसे सार्वजनिक दिया। ” तो चलो देखते कि क्या हमें अधिकार मिला है। एफबीआई निदेशक शैफेट और जीओपी समिति अध्यक्षों संभावित रूप राजनीतिक रूप विस्फोटक जांच एक प्रमुख विकास बारे बताता  और न शैफेट और सहयोगियों डेमोक्रेटिक समकक्षों बारे बताने शिष्टाचार था। बजाय  सहयोगी अनुसार  ट्विटर बारे पता लगाया। डेली कोस बात चुकी कि कॉमी खुद शैफेट और रिपब्लिकन पत्र अग्रिम नोटिस  जिससे स्पिन मशीन चालू समय मिल गया। कि अच्छे थिएटर बना  ऐसा नहीं जो बताता कि मामला है। आखिरकार  ऐसा नहीं जो बताता कि कॉमी घोर अक्षम और टोन-बहरा अलावा नहीं था। हालांकि  सुझाव देता कि चैफेट्ज़ एक अभिनय  जो डैन बर्टन और डेरेल इस्सा ज़िम्मेदारी और द्विदलीयता मॉडल दिखता है। पास विस्फोटक बारे रैंकिंग सदस्य एलिजा कमिंग्स सूचित शालीनता नहीं थी। निष्पक्षता बुनियादी मानकों नहीं रौंदता  तो मुझे नहीं पता कि क्या है। दी गई  संभावना नहीं कि शैफेट जवाब देना होगा। प्रोवो और ओरेम लंगर डाले एक हास्यास्पद रिपब्लिकन जिले बैठता ; + कुक पार्टिसन वोटिंग इंडेक्स  और मिट रोमनी प्रतिशत वोट दंड था। अलावा  रिपब्लिकन हाउस नेतृत्व शैफेट योजनाबद्ध मछली पकड़ने अभियान पूर्ण समर्थन है। मतलब नहीं कि हम उस गर्म रोशनी नहीं डाल सकते। आखिरकार  एक पाठ्यपुस्तक उदाहरण जो सदन रिपब्लिकन नियंत्रण है। और दुनिया दूसरा बुरा व्यक्ति है। डेरेल लुकस बारे डारेल उत्तरी कैरोलिना विश्वविद्यालय स्नातक जो खुद पुराने स्कूल पत्रकार मानता है। कॉलेज धार्मिक अधिकार एक सदस्य रूप बदलने प्रयास केवल धार्मिक अधिकार बुरे सपने बदलने सफल - एक करिश्माई ईसाई जो एक अप्रकाशित उदारवादी है। उन लोगों खड़े इच्छा जो केवल चुप्पी डर गए  एक अपमानजनक तीन साल शादी बच गए। ईपू  डेली कोस ईसाई मांग रूप जान हैं। ट्विटर  अनुसरण फेसबुक साथ जुड़ें। खरीदने यहां क्लिक करें। जुडिये"])

array([1])

In [ ]:
!pip install joblib
import joblib
joblib.dump(pipeline_svc, '/content/drive/MyDrive/NLP Project/news_classifier.pkl')
# joblib.dump(pipeline_rfc_sent,'/content/drive/MyDrive/NLP Project/sentiment_classifier.pkl')

['/content/drive/MyDrive/NLP Project/news_classifier.pkl']

In [ ]:
zero_sentiment_rows = train[train['sentiment'] == 0]
zero_sentiment_rows.text

18      हिलेरी एफबीआई बंद ! होम » मुख्य समाचार » विश्व...
19      बुधवार न्यूयॉर्क शहर ट्रम्प टॉवर डोनाल्ड ट्रम्...
23      मैसाचुसेट्स कॉप पत्नी ब्लैक लिव्स मैटर स्रोत प...
52      एक नए अध्ययन पाया कि % सड़क किनारे दुर्घटनाओं ...
54      सामन्था बी अमेरिकी राजनीति ( वीडियो ) ऑल्ट राइ...
                              ...                        
2055    ओबामा चुनाव प्रभावित वामपंथी समूहों धन स्थानां...
2068    वीं सदी तार हैं… महान मुख्यधारा मीडिया और डेमो...
2074    एक तर्क ट्रम्प समर्थकों और रूढ़िवादी बना कि फॉ...
2075    राष्ट्रपति डोनाल्ड ट्रम्प कार्यकारी आदेश अस्था...
2089                   हमारा समर्थन आगमन बोस्नियाई अनुवाद
Name: text, Length: 246, dtype: object